# Algorithms for Flowpipe Approximation using SOS 

### Marcelo Forets, Universidad de la República, Uruguay.

#### ICCOPT 2019, TU-Berlin, August 5th, 2019

## Outline

- Context

- From Hamilton-Jacobi to SOS optimization

- Results
  - Validation of the implementation
  - Sogokon's benchmarks

- Conclusions and future work

## Context

### What are *dynamical systems*?

Just about anything that evolves with time!

- either discrete time or continuous time
- .. or both: *hybrid* i.e. continuous time with discrete transitions or jumps
- several mathematical formalisms available: ODEs, DAEs, PDEs, hybrid automata

<img src="ICCOPT2019/fig/quadrotor2.png" width=550>

<font size="-1">Immler, F., Althoff, M., Benet, L., Chapoutot, A., Chen, X., Forets, M., ... & Schilling, C. (2019). <b> ARCH-COMP19 Category Report: Continuous and Hybrid Systems with Nonlinear Dynamics.</b> EPiC Series in Computing, 61, 41-61.</font>

### Application domains

- Cyber-physical system: <font color='blue'>a device that by means of computation is able to control or interact with a physical process</font>
- Examples:
    - embedded controllers for aircrafts, autonomous cars
    - medical devices
    - other safety-critical or mission-critical applications

<img src="ICCOPT2019/fig/CPS.png" width=600>

### The problem

- Complex, real-world systems are prone to *failures*
- "[...] industrial standards, such as <font color='red'>  ISO 26262, ISO 61508, IEC 62304, EN 50128 explicitly recommend the use of formal methods </font> in the design and development of autonomous systems"(*excerpt from FT4DAS 2019: 1st Workshop on Formal Techniques for Dependable Autonomous Systems*)

<img src="ICCOPT2019/fig/CPSfail.png" width=600>

- Engineers need *safety* and *reliability* guarantees to take design decisions under non-deterministic inputs, parameters or noise

$\Longrightarrow$<font color='green'> The field of **reachability analysis** is concerned with understanding the set of all possible behaviors of such systems.

### Methods overview 

- Set-based methods
    - Template polyhedra: polytopes / support functions, zonotopes
    - Interval methods
    - Taylor models
    - Ellipsoidal methods

- Optimisation: 
    - <font color='red'> Hamilton-Jacobi PDE:</font> Convex programming, Level set methods
    - Barrier certificates

(See https://github.com/JuliaReach/Reachability.jl/wiki/Related-tools for a list of +25 reachability analysis tools!)

- <font size="-1"> Bogomolov, S., Forets, M., Frehse, G., Potomkin, K., & Schilling, C. (2019, April). <b>JuliaReach: a toolbox for set-based reachability. </b>In Proceedings of the 22nd ACM International Conference on Hybrid Systems: Computation and Control (pp. 39-44). ACM.</font> 
- <font size="-1">Immler, F., Althoff, M., Benet, L., Chapoutot, A., Chen, X., Forets, M., ... & Schilling, C. (2019). <b> ARCH-COMP19 Category Report: Continuous and Hybrid Systems with Nonlinear Dynamics.</b> EPiC Series in Computing, 61, 41-61.</font>
- <font size="-1">Althoff, M., Bak, S., Chen, X., Fan, C., Forets, M., Frehse, G., ... & Schilling, C. (2018). <b> ARCH-COMP18 Category Report: Continuous and Hybrid Systems with Linear Continuous Dynamics.</b> In Proc. of the 5th International Workshop on Applied Verification for Continuous and Hybrid Systems (pp. 23-52).</font>

## Theory

### General (informal) idea

<img src="ICCOPT2019/fig/flow.png" align="center" width=330>

- Find a PDE that describes the <font color='green'> *flow*</font> of the dynamics (think of a viscous fluid).
- Search for an approximate analytical polynomial solution to this PDE using a hierarchy of convex programs (Lasserre's hierarchy), consisting of LMIs and constructed by SOS decomposition techniques.
- The zero sub-level sets converge to the exact reachable set from the inside in measure (without discretizing state-space).

### Theoretical work (not exhaustive)

- **[KHJ13]** M. Korda, D. Henrion, C. N. Jones. *Inner approximations of the region of attraction for polynomial dynamical systems.* IFAC Proceedings Volumes, 46(23):534-539, 2013.

- **[MBT06]** Mitchell, Ian M., Alexandre M. Bayen, and Claire J. Tomlin. *A time-dependent Hamilton-Jacobi formulation of reachable sets for continuous dynamic games.* IEEE Transactions on automatic control 50.7 (2005): 947-957.

- **[MGHT]** Magron, V., Garoche, P. L., Henrion, D., & Thirioux, X. (2017). *Semidefinite approximations of reachable sets for discrete-time polynomial systems.* arXiv preprint:1703.05085. To appear in SIAM Journal on Control and Optimization.

- **[XFZ18]** Xue, Bai, Martin Fränzle, and Naijun Zhan. *Under-approximating reach sets for polynomial continuous systems.* Proceedings of the 21st International Conference on Hybrid Systems: Computation and Control (part of CPS Week). ACM, 2018.

- **[LAS01]** Lasserre, J. B. (2001). *Global optimization with polynomials and the problem of moments.* SIAM Journal on optimization, 11(3), 796-817.

## Running assumptions

- A continuous dynamical system:

<font color='blue'>
\begin{equation}
\dfrac{dx}{dt} = f(x),\qquad x_0 \in \mathcal{X}_0,~~t \in [t_0, T]
\end{equation}
</font>

- Initial set of states (assumed compact) is basic semialgebraic:

<font color='blue'>
\begin{equation}
\mathcal{X}_0 := \{ x \in \mathbb{R}^n : V_0(x) \leq 0\}
\end{equation}
</font>

- Compact search space $\mathcal{Y} \times [t_0, T]$ where:

<font color='blue'>
\begin{equation}
\mathcal{Y} := \{ x \in \mathbb{R}^n : g(x) \geq 0\}
\end{equation}
</font>

- <font color='red'> Goal:</font> obtain under or overapproximations of the exact reachable set, denoted $\Omega(t; \mathcal{X}_0, t_0)$ and defined as:

<font color='blue'>
\begin{equation}
\Omega(t; \mathcal{X}_0, t_0) := \{ x : x_0 \in \mathcal{X}_0 \wedge x = \phi(t; x_0, t_0)\},
\end{equation}
</font>

### Method: Hamilton-Jacobi equations

- The associated HJE is:

<font color='blue'>
\begin{equation}
\mathcal{L}_f\Phi(x, t) := \dfrac{\partial \Phi(x, t)}{\partial t} + \dfrac{\partial \Phi(x, t)}{\partial x} f(x) = 0,\qquad \Phi(x, t_0) = V_0(x).
\end{equation}
</font>

- The exact reachable set can be obtained from the associated HJE (**[MBT06]**).

- The reachable set at time $t$ is:

<font color='blue'>
\begin{equation}
\Omega(t; \mathcal{X}_0, t_0) = \{x : \Phi(x, t) \leq 0\}.
\end{equation}
</font>

- Under regularity assumptions, there exists a unique classical solution $\Phi(x, t)$ in $[t_0, T]$.

### Reachable set computations

- Idea: Relax the associated HJ equation $\Rightarrow$ solve a semidefinite program

- An <font color='red'> underapproximation </font> can be constructed by the zero-sublevel set of a continuously differentiable function solving a system of constraints derived by relaxing the HJE.

**Theorem [Theorem 1, XFZ18].** Under the running assumptions, suppose that a continuously differentiable function $\Phi(x, t)$ and $\epsilon \in \mathbb{R}$ satisfy:

\begin{eqnarray}
&&\epsilon \ge 0\\
&&0 \leq \mathcal{L}\Phi(x) \leq \epsilon,\qquad \forall (x, t) \in \mathbb{R}^n\times[t_0, T]\\
&&\Phi(x, t_0) \geq V_0(x)\\
&&\Phi(x, t_0) - \epsilon \leq V_0(x)\\
\end{eqnarray}
Then the set $\{x : \Phi(x, \tau) \leq 0\}$ is an <font color='red'> underapproximation </font> of the exact reachable set for $\tau \in [t_0, T]$.

### Reachable set computations

- An <font color='red'> overapproximation </font> can be employed to quantitatively characterize how close the under-approximation is to the exact reachable set:

**Theorem [Corollary 1, XFZ18].**

The set $\{x : \Phi(x, \tau) \leq \epsilon(\tau - t_0 + 1)\}$ is an overapproximation of the exact reachable set for $\tau \in [t_0, T]$.

- $\epsilon \geq 0$ quantifies the <font color='green'> quality </font> of the approximation (the smaller the better).

### Relaxation to the search region

- Let $\mathcal{Y} := \{ x \in \mathbb{R}^n : g(x) \geq 0\}$ be an appropriate <font color='green'> search region </font> (trajectories shouldn't escape).
 
Compute $\epsilon_k^* = \min \epsilon$ over the tuple $(\epsilon, \Phi(x, t))$, subject to the constraints:

\begin{eqnarray}
&& \epsilon \geq 0 \\
&& \mathcal{L}\Phi(x, t) \geq 0,\qquad \forall (x, t) \in \mathcal{Y}\times [t_0, T] \\
&&\epsilon - \mathcal{L}\Phi(x) \geq 0,\qquad \forall (x, t) \in \mathcal{Y}\times[t_0, T]\\
&&\Phi(x, t_0) - V_0(x) \geq 0,\qquad \forall x \in \mathcal{Y}\\
&&V_0(x) + \epsilon - \Phi(x, t_0) \geq 0,\qquad \forall x \in \mathcal{Y}\\
\end{eqnarray}
Then the set $\{x : \Phi(x, \tau) \leq 0\}$ is an <font color='red'> underapproximation </font> of the exact reachable set for $\tau \in [t_0, T]$.

- Such relaxation does not introduce conservativeness.

### Formulation of the SOS

- The following SDP relaxation holds:

Compute $\epsilon_k^* = \min \epsilon$ over the tuple $(\epsilon, \Phi_k(x, t), \{s_i\}_{i=0}^9\}$ where $k$ is the relaxation order, and subject to the constraints:

\begin{eqnarray}
&& \epsilon \geq 0 \\
&& \mathcal{L}\Phi_k(x, t) = s_0 + s_1(t-t_0)(T-t)+s_2g(x) \\
&& \epsilon - \mathcal{L}\Phi_k(x, t) = s_3 + s_4(t-t_0)(T-t) + s_5g(x)\\
&&\Phi_k(x, t_0) - V_0(x) = s_6 + s_7g(x_0)\\
&&\epsilon + V_0(x) - \Phi_k(x_0, t_0) = s_8 + s_0g(x)
\end{eqnarray}

- SOS poly. constr. can be written explicitly as LMIs and the objective is linear in $\epsilon$ $\Rightarrow$
  - the solution can be obtained with an  <font color='green'> SDP solver </font> 
  - solvable in <font color='green'> polynomial time </font> to any desired accuracy
- The problem gives rise to a <font color='green'> converging sequence</font> of inner-approximations of $\Omega$ in measure (for proofs see [XFZ18]), i.e. <font color='green'>  $\epsilon \to 0$ as $k \to \infty$.</font> 

### A (very) quick introduction Taylor model flowpipe approximation

<img src="ICCOPT2019/fig/mu1_zono.png" align="center" width=500>

## Validation

## Why Julia?

- General-purpose programming language, <font color='green'>*designed* for scientific computing</font>

- *Dynamically-typed*, *high level* and *fast*:
    - Types are *run-time* objects which can *convey* information to the compiler
    - User-defined types are as fast as built-in
    - <font color='green'>*Composability*</font>

- <font color='red'>Free/Libre</font>, open source with MIT license: **open science**  

- https://julialang.org

The implementation requires <font color='green'>Julia ≥ v1.0</font>, and involves:

### Optimisation:

- `JuMP.jl` (<font color='blue'>I. Dunning, J. Huchette, M. Lubin, B. Legat et al</font>)
  - Modeling language for Mathematical Optimization
- `SumOfSquares.jl` (<font color='blue'>B. Legat, C. Coey, L. Kapelevich, J. P. Vielma, T. Weisser</font>)
  - Sum of Squares Programming extension for JuMP
  
### Reachability API:

- `Reachability.jl` (<font color='blue'>M. Forets, C. Schilling, L. Benet</font>)
  - Reachability analysis, safety properties
- `MathematicalSystems.jl` (<font color='blue'>M. Forets, C. Schilling, B. Legat</font>)
  - Julia package for mathematical systems interfaces

### For set-based methods:

- `LazySets.jl` (<font color='blue'>M. Forets, C. Schilling</font>)
  - Set representations and operations for convex and non-convex sets, approximations

- `IntervalArithmetic.jl` (<font color='blue'>L. Benet, D.P. Sanders</font>)
  - Rigorous floating-point calculations using interval arithmetic in Julia
- `TaylorSeries.jl` (<font color='blue'>L. Benet, D.P. Sanders</font>)
  - Taylor series for one and several variables
- `TaylorModels.jl` (<font color='blue'>L. Benet, D.P. Sanders</font>)
  - Taylor series for one and several variables
- `TaylorIntegration.jl` (<font color='blue'>L. Benet, J.A. Pérez-Hernández</font>)
  - Taylor method to integrate ODEs (initial value problems); jet-transport techniques

### Van der pol model

- Introduced by the Dutch physicist Balthasar van der Pol (1926).

- Defined by the following ODE with 2 variables:

\begin{eqnarray}
x' &=& y\\
y'  &=& \mu (1-x^2)y - x\\
\end{eqnarray}

- It is a nonconservative stable oscillator.
- The system has a stable limit cycle however that becomes increasingly sharp for higher values of $μ$.

### Comparison with an implementation in MATLAB/YALMIP

- [YALMIP](https://yalmip.github.io/) is a well-known MATLAB based modelling language, suitable for a large class of problems in control theory and optimization.
- see [YALMIP: A toolbox for modeling and optimization in MATLAB](https://ieeexplore.ieee.org/document/1393890), 2004 IEEE ICRA


```matlab
sdpvar x y t obj;
f = 2*[y; (-0.2*x+y-0.2*x^2*y); 0.5];
T = 1;
V0 = x^2+y^2-0.25;
g = 25-x^2-y^2;
degree = 12;

x1 = [x y t];
[Phi, coe] = polynomial(x1,degree);
Phi_0 = replace(Phi,[t],[0]);
Phi_derivative = jacobian(Phi,x1)*f;
 
[s1,coe1] = polynomial(x1,degree);
[s2,coe2] = polynomial(x1,degree);
[s4,coe4] = polynomial(x1,degree);
[s5,coe5] = polynomial(x1,degree);
[s7,coe7] = polynomial([x y],degree);
[s9,coe9] = polynomial([x y],degree);

F = [sos(Phi_derivative-s1*t*(T-t)-s2*g),sos(obj-Phi_derivative-s4*t*(T-t)-s5*g),sos(Phi_0-V0-s7*g),...
     sos(obj+V0-Phi_0-s9*g),sos(s1),sos(s2),sos(s4),sos(s5),sos(s7),sos(s9),obj>=0];
 
ops = sdpsettings('solver','mosek','sos.newton',1,'sos.congruence',1);
diagnostics= solvesos(F,obj,ops,[obj;coe;coe1;coe2;coe4;coe5;coe7;coe9])
```

22 LOC

```julia
@polyvar x₁ x₂ t
f = 2 * [x₂, -0.2*x₁ + x₂ - 0.2*x₁^2*x₂] 
T = 1.0 
V₀ = x₁^2 + x₂^2 - 0.25
g = 25 - x₁^2 - x₂^2
k = 12

X = monomials([x₁, x₂], 0:k)
XT = monomials([x₁, x₂, t], 0:k)

model = SOSModel(with_optimizer(Mosek.Optimizer))
@variable(model, Φ, Poly(XT))

∂xf = α -> ∂(α, x₁) * f[1] + ∂(α, x₂) * f[2] 
LΦ = ∂(Φ, t) + ∂xf(Φ)

Φ₀ = subs(Φ, t => 0.)
@variable(model, ϵ)
dom1 = @set t*(T-t) >= 0 && g >= 0
dom2 = @set g >= 0
@constraint(model, ϵ >= 0.)
@constraint(model, LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, ϵ - LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, Φ₀ - V₀ ∈ SOSCone(), domain = dom2)
@constraint(model, ϵ + V₀ - Φ₀ ∈ SOSCone(), domain = dom2)

@objective(model, Min, ϵ)
optimize!(model)
```
23 LOC

|  Package    | k    |Constraints|Scalar variables|Matrix variables|Time(s)|
|-------------|------|-----------|----------------|----------------|-------|
|**SumOfSquares v0.3.0**         |    2 |    83    |      13       |            8   |   < 1 |
|YALMIP       |    2 |       152 |             63 |              10|   < 1 |
||
|**SumOfSquares v0.3.0**         |    3 |    199    |        21     |        10       |   < 1 |
|YALMIP       |    3 |       254 |            121 |              10|   ~ 1 |
||
|**SumOfSquares v0.3.0**         |    4 |   199     |       36      |      10         |   < 1 |
|YALMIP       |    4 |       394 |           206  |              10|  1.18 |
||
|**SumOfSquares v0.3.0**         |    5 |     387   |       57      |         10      |   < 1 |
|YALMIP       |    5 |      578  |         323     |          10    |  0.11 |
||
|**SumOfSquares v0.3.0**         |    6 |    387    |       85      |       10        |   < 1 |
|YALMIP       |    6 |    812    |         477    |    10          |  1.10  |
||
|**SumOfSquares v0.3.0**         |    7 |   663     |     121        |      10         |   < 1 |
|YALMIP       |    7 |     1102   |        673     |        10      |  1.52 |


|  Package    | k    |Constraints|Scalar variables|Matrix variables|Time(s)|
|-------------|------|-----------|----------------|----------------|-------|
|**SumOfSquares v0.3.0**         |    8 |  663      |        166     |           10    |   < 1 |
|YALMIP       |    8 |    1454    |       916       |          10    |  1.10 |
||
|**SumOfSquares v0.3.0**         |    9 |      1043  |        221     |        10       |   1.70 |
|YALMIP       |    9 |   1874     |       1211     |         10     |  1.58  |
||
|**SumOfSquares v0.3.0**         |    10 |    1043    |    287         |     10          |   1.67 |
|YALMIP       |   10 |     2368   |       1563     |        10      |  2.73 |
||
|**SumOfSquares v0.3.0**         |    11 |   1543     |      365       |     10          |   4.88 |
|YALMIP       |   11 |   2942    |       1977      |       10       |  2.30 |
||
|**SumOfSquares v0.3.0**         |    12 | 1543       |   456          |         10      |  5.02 |
|YALMIP       |   12 |    3602    |      2458       |      10        | 6.57  |


- Comparison of the *size* of the generated model and the *runtime* for Van der Pol
- Using *Mosek* solver solver in both cases

### Real roots of $\Phi(x, t)$ at $t = 2.0$

<p> <img src="ICCOPT2019/fig/k_12_b.png" width=500> </p>

|Relax. order k|$\epsilon$|
|---|----|
|10|0.72|
|11|0.038|
|12|0.072|
|13|0.006|
|14|0.003|

- computed using `HomotopyContinuation.jl`, see https://github.com/JuliaHomotopyContinuation

### Comparison with TMJets (ARCHCOMP-2019 specification)

<p> <img src="ICCOPT2019/fig/tm_1.png" width=600> </p>

- $x_0 \in [1.25, 1.55], y_0 \in [2.35, 2.45]$
- <font color='red'> specification: $y < 2.75$ for all $t \in [0, 7]$ </font>
- SOS: very slow convergence! - `MOSEK_STATUS: SLOW_PROGRESS`

##  Benchmarks

### What to benchmark?

- Aim: Assess reliability, efficiency and validation of known methods.
- Benchmarks should have diverse properties so that can be truly useful to test methods in an <font color='red'> unbiased way </font>.
- Two groups:
   - "Test" examples, common standard benchmarks from the literature
   - Real-world problems (physics, chemistry, economics, etc)

<font size="-1"> Jamil, Momin, and Xin-She Yang. <b> A literature survey of benchmark functions for global optimization problems.</b> arXiv preprint arXiv:1308.4008 (2013).</font>

### Sogokon's benchmarks

- Originally safety-verification benchmarks (in unbounded time)
- 65 benchmarks for nonlinear polynomial ODEs

<img src="ICCOPT2019/fig/flow.png" align="center" width=330>

- (Dai et al., Barrier certificates)

\begin{eqnarray}
&& x' = 2x - xy \\
&& y' = 2x^2 - y\\
\end{eqnarray}

<font size="-1"> Sogokon, Andrew, Khalil Ghorbal, and Taylor T. Johnson. <b> Non-linear continuous systems for safety verification (benchmark proposal).</b> 2016.</font>

### Work-precision

|Relax. order k|$\epsilon$|Runtime (sec)|
|---|----|----|
|15|2.78|15|
|16|1.85|36|
|17|1.54|38|
|18|1.42|78|

##  Conclusions

### To recap

- The behavior of non-linear systems is extremely difficult to analyze because solutions are rarely available in closed-form.
- The exact computation of reachable sets of an <font color='green'>uncertain IVP for a nonlinear system</font> is generally impossible, and we use underapproximative or overapproximative methods for property checking in safety-critical systems.
- Numerical experiments are essential to understand the extent to which available methods and solvers are applicable.

### Pros and Cons

- Convex programming offers an interesting approach, <font color='red'>however:</font>
  - Generally <font color='red'> slower converge rates </font> w.r.t. set-based methods
  - Hard to know appropriate (i.e. default) <font color='red'> problem-specific options </font>(eg. relaxation degree, search region) to get a predefined
    accuracy
- Pros:
  - Exploit *polynomial* structure of the problem and <font color='green'> can handle polynomial constraints exactly </font>
  - Growing ecosystem allows to <font color='green'> easily formulate and solve </font> such problems (`JuMP`, `SumOfSqures.jl`, ...) and compare with other methods (using `JuliaReach`)

### Future work

- combination of SOS and set-based methods
- exploit more $\Phi(x, t)$, eg. through ICP or parametric root finding
- extension to *hybrid systems*
- systems with uncertain parameters

### Acknowledgements

- JuliaReach development is led by:
  - Marcelo Forets (Univ. de la República, Uruguay)
  - Christian Schilling (Institute of Science and Technology, Austria)

- Contributors to JuliaReach open projects include:
  - A. Deshmuhk (Indian Institute of Information Technology, India), B. Garate (Univ. de la República, Uruguay),
    S. Guadalupe (Univ. de la República, Uruguay), N. Kekatos (Univ. Grenoble Alpes, France), B. Legat
    (UCLouvain, Belgium), K. Potomkin (ANU, Australia), A. Rocca (INRIA, France), F. Viry (CERFACS, France)

- Scientific collaborators:
    - L. Benet (UNAM, México), S. Bogomolov (ANU, Australia), G. Frehse (ENSTA ParisTech, France),
      A. Podelski    (Univ. of Freiburg, Germany), David P. Sanders (UNAM, México)

---


<font color='blue'> Join the effort!  </font>https://github.com/juliareach https://gitter.im/JuliaReach/Lobby

<font color='green'> Contact: mforets@gmail.com </font> http://github.com/mforets